# Recipe
## Pick Time with Courier

In [7]:
# setup for training AND prediction
import analitico
import analitico.plugin
import s24.plugin

from analitico.pandas import *

import numpy as np
from scipy import stats

# pass api token to create factory
factory = analitico.authorize("tok_demo1_croJ7gVp4cW9")

def sample(df):
    return df.sample(n=4)

import datetime
print(datetime.datetime.now())

2019-04-11 15:02:31.178320


In [8]:
import missingno as msno
%matplotlib inline
import matplotlib.pyplot as plt

import pandas as pd
pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [9]:
%config Application.log_level='DEBUG'
%config Application.log_level='INFO'
import logging
logging.getLogger().setLevel(logging.INFO)
log = logging.getLogger()
log.debug('Test debug')
log.info('Test info')

In [10]:
action = "train"

y_label = "pick_time.min"

# data used for sample prediction
data = [{
        "order_fulfillment_type": 1,
        "order_deliver_at_start": "2018-08-27 19:00:00",
        "courier_id": 109135.0,
        "odt_items_total": 12,
        "odt_items_with_variable_weight": 0,
        "store_name": "esselunga",
        "store_area": "MI6",
        "store_province": "MI",
        "store_ref_id": 5042,
        "courier_area": "MI13",
        "courier_orders_taken": 249,
        "courier_experience_days": 306.2845833333,
        "pick_time.min": 10.2 # just for testing, real data would not have y_true of course
    },
    {
        "order_fulfillment_type": 1,
        "order_deliver_at_start": "2018-09-28 20:00:00",
        "courier_id": 112147.0,
        "odt_items_total": 16,
        "odt_items_with_variable_weight": 1,
        "store_name": "ipercoop",
        "store_area": "MI4",
        "store_province": "MI",
        "store_ref_id": 4494,
        "courier_area": "MI4",
        "courier_orders_taken": 3387,
        "courier_experience_days": 316.7805439815,
        "pick_time.min": 38.9 # just for testing, real data would not have y_true of course
    }]



## Load training data

In [11]:
orders = factory.run_plugin(action=action, settings = {
    "name": "analitico.plugin.DatasetSourcePlugin",
    "dataset_id": "ds_s24_order_times_dyn"
})
sample(orders)

,order_amount,order_volume,order_deliver_at_start,order_deliver_at_end,order_paid_at,order_delivered_at,order_fulfillment_type,store_id,courier_id,picker_id,customer_id,odt_items_total,odt_items_with_variable_weight,odt_first_touched_at,odt_last_touched_at,store_name,store_area,store_province,store_lat,store_lng,store_ref_id,customer_province,customer_lat,customer_lng,customer_area,customer_ztl,customer_ref_id,customer_created_at,courier_soldo_enabled,courier_area,courier_orders_taken,courier_orders_sent,courier_created_at,courier_experience_days,pick_time.min,pay_time.min,deliver_time.min
168336,40.45,8.89,2019-02-15 14:00:00,2019-02-15 15:00:00,2019-02-15 14:15:31,2019-02-15 14:38:10,1,1005616,155657.0,NaN,1005652,6,0,2019-02-15 14:01:45,2019-02-15 14:06:52,coop,MI13,MI,45.370128,9.212598,4431,MI,45.339928,9.085551,MI13,0,0001432796,2019-02-14 00:51:23,0,NaN,0,0,2019-04-11 14:25:10.676079,-55,5.116667,8.650000,22.650000
113645,59.18,115.15,2018-12-15 18:00:00,2018-12-15 19:00:00,2018-12-15 17:27:22,2018-12-15 17:42:59,1,885005,152809.0,NaN,885041,30,4,2018-12-15 16:37:39,2018-12-15 17:11:59,coop,MI15,MB,45.615118,9.201361,5041,MI,45.624449,9.167875,MI15,0,0001362129,2018-12-15 15:08:49,0,NaN,0,0,2019-04-11 14:25:10.676079,-116,34.333333,15.383333,15.616667
183762,80.88,220.85,2019-03-01 16:00:00,2019-03-01 17:00:00,2019-03-01 16:10:24,2019-03-01 16:31:59,1,1038860,101498.0,NaN,1038896,36,4,2019-03-01 15:33:03,2019-03-01 16:04:37,lidl,RM6,RM,41.894730,12.591020,5270,RM,41.879372,12.572047,RM6,0,0001118723,2019-03-01 14:15:35,0,NaN,0,0,2019-04-11 14:25:10.676079,-40,31.566667,5.783333,21.583333
51357,85.42,141.30,2018-10-01 19:00:00,2018-10-01 20:00:00,2018-10-01 18:49:29,2018-10-01 19:31:38,1,746201,142677.0,NaN,746237,28,4,2018-10-01 17:36:10,2018-10-01 18:36:59,famila superstore,VR1,VR,45.462180,10.968289,2769,VR,45.445518,11.005071,VR1,0,0000728159,2018-10-01 10:12:13,0,NaN,0,0,2019-04-11 14:25:10.676079,-191,60.816667,12.500000,42.150000


In [12]:
#plugin = s24.plugin.AugmentCouriersPlugin(factory=factory)
orders = plugin.run(orders, action=action)

# Drop all the columns that will not be needed for this model
orders.drop(
    [
        "order_id", "order_deliver_at_end", "order_paid_at", "order_delivered_at", "order_amount", "order_volume",
        "customer_id", "customer_ref_id", "customer_province", "customer_lat", "customer_lng", "customer_area", "customer_ztl", "customer_created_at",
        "store_id", "store_lat", "store_lng",
        "picker_id",
        "courier_created_at", "courier_soldo_enabled", "courier_orders_sent",
        "odt_first_touched_at", "odt_last_touched_at",
        "pay_time.min", "deliver_time.min"
    ],
    axis = 1,
    inplace = True
)

AugmentCouriersPlugin: data is already augmented


KeyError: "['order_id'] not found in axis"

In [ ]:
print("Total orders: %d" % len(orders))
# orders completed in less than 2 minutes are bogus and should be removed
orders = orders[orders["pick_time.min"].notnull()]
orders = orders.loc[orders["pick_time.min"] > 2]
orders = orders.loc[orders["pick_time.min"] < 120]
print("Orders between 2 and 120 minutes: %d" % len(orders))

In [ ]:
# check for missing data
#msno.matrix(orders)
pd_print_nulls(orders)

In [ ]:
# distribution of values BEFORE filtering outliers
columns = ["pick_time.min", "odt_items_total", "odt_items_with_variable_weight", "courier_orders_taken", "courier_experience_days"]
orders[columns].describe()

In [ ]:
def chart_distributions():
    fig, axes = plt.subplots(1, 5, figsize=(16, 3))
    hist = orders[["pick_time.min"]].hist(bins=range(0,120,1), ax=axes[0])
    hist = orders[["odt_items_total"]].hist(bins=range(0,120,1), ax=axes[1])
    hist = orders[["odt_items_with_variable_weight"]].hist(bins=range(0,30,1), ax=axes[2])
    hist = orders[["courier_orders_taken"]].hist(bins=range(0,1000,25), ax=axes[3])
    hist = orders[["courier_experience_days"]].hist(bins=range(0,300,7), ax=axes[4])
chart_distributions()

In [ ]:
def pd_remove_outliers(df, column, zscore):
    df1 = df[[column]]
    return df[(np.abs(stats.zscore(df1)) <= zscore).all(axis=1)]

# keep features within n standard deviations
orders = pd_remove_outliers(orders, "pick_time.min", 3)
orders = pd_remove_outliers(orders, "odt_items_total", 3)
print("Filtered orders: %d" % len(orders))

In [ ]:
# distribution of values AFTER filtering outliers
columns = ["pick_time.min", "odt_items_total", "odt_items_with_variable_weight", "courier_orders_taken", "courier_experience_days"]
orders[columns].describe()

In [ ]:
chart_distributions()

In [ ]:
# augment delivery window date
orders = factory.run_plugin(orders, action=action, settings={
    "name": "analitico.plugin.AugmentDatesPlugin",
    "schema": {
        "columns": [
            { "name": "order_deliver_at_start" }
        ]
    }
})
#orders.drop(["order_deliver_at_start"], axis=1, inplace=True)
sample(orders)

In [ ]:
# check a few orders
orders.tail(15)

In [ ]:
# check data types
orders["courier_id"] = orders["courier_id"].astype("category")
orders["order_fulfillment_type"] = orders["order_fulfillment_type"].astype("category")
orders["store_name"] = orders["store_name"].astype("category")
orders["store_area"] = orders["store_area"].astype("category")
orders["store_province"] = orders["store_province"].astype("category")
orders["store_ref_id"] = orders["store_ref_id"].astype("category")
orders["courier_area"] = orders["courier_area"].astype("category")
orders["store_ref_id"] = orders["store_ref_id"].astype("category")
orders.dtypes

In [ ]:
# train model with gradient boosting on decision trees
training = factory.run_plugin(orders, action="train", settings = {
    "name": "analitico.plugin.CatBoostRegressorPlugin",
    "data": {
        "chronological": False,
        "label": "pick_time.min"
    },
    "parameters": {
        "learning_rate": 1,
        "iterations": 100,
        "test_size": 0.2
    }
})

In [ ]:
training

In [ ]:
training["scores"]["features_importance"]

In [ ]:
features_importance = training["scores"]["features_importance"]

features = list(features_importance.keys())
importances = list(features_importance.values())
indices = np.argsort(importances)[::-1]

# Rearrange feature names so they match the sorted feature importances
names = [features[i] for i in indices]

# Create plot
plt.figure()

# Create plot title
plt.title("Feature Importance")

# Add bars
num = len(features_importance)
plt.barh(range(0, num), importances)
#plt.bar([0,1,2], [3,30,90])

# Add feature names as x-axis labels
#plt.xticks(range(0, num), features, rotation=90)
plt.yticks(range(0, num), features)
plt.gca().invert_yaxis()
# Show plot
plt.show()

In [ ]:
## Prediction

In [ ]:
samples_filename = os.path.join(factory.get_artifacts_directory(), "training-samples.json")
data = pd.read_json(samples_filename, orient="records")
data

In [ ]:
if not isinstance(data, pd.DataFrame):
    data = pd.DataFrame.from_dict(data, orient="columns")

# when 
if not "order_deliver_at_start" in data.columns:
    data["order_deliver_at_start"] = pd.Timestamp.now()
    
data

In [ ]:
# generate predictions
results = factory.run_plugin(data, action="predict", settings = [
    {
        "name": "s24.plugin.AugmentCouriersPlugin"
    },
    {
        "name": "analitico.plugin.AugmentDatesPlugin",
        "schema": {
            "columns": [
                { "name": "order_deliver_at_start" }
            ]
        }
    },
    {
        "name": "analitico.plugin.CatBoostRegressorPlugin",
        "data": {
            "chronological": False,
            "label": "pick_time.min"
        },
        "parameters": {
            "learning_rate": 1,
            "iterations": 100,
            "test_size": 0.2
        }
    }
])

In [ ]:
results

In [ ]:
data

In [ ]:
# print test results if running entire notebook linearly
from sklearn.metrics import *

if y_label in data.columns and not data[y_label].isnull().values.any():
    y_true = data[y_label]
    y_pred = results["predictions"]

    print("mean_squared_error: %.3f" % mean_squared_error(y_true, y_pred))
    print("mean_absolute_error: %.3f" % mean_absolute_error(y_true, y_pred))
    print("median_absolute_error: %.3f" % median_absolute_error(y_true, y_pred))
    print("explained_variance_score: %.3f\n" % explained_variance_score(y_true, y_pred))

    print("row  items   y_true   y_pred  true/pr")
    for i in range(len(data.index)):
        items = data.loc[i]["odt_items_total"]
        print("%3d %6d %8.2f %8.2f %8.2f" % (i, items, y_true[i], y_pred[i], 1-(y_true[i] / y_pred[i])))